In [1]:
import pandas as pd
import polars as pl
import utils
from datetime import datetime

In [2]:
def data_info(df):
    print(df.shape)
    # 查看过滤后的 DATE 范围（以整数形式获取最小和最大值）
    min_date = df.select(pl.col("DATE").min()).to_series()[0]
    max_date = df.select(pl.col("DATE").max()).to_series()[0]
    print(f"\n过滤后的数据 DATE 范围（整数格式）：最小值 = {min_date}, 最大值 = {max_date}")

    # 如果希望将整数格式转换为 datetime 格式，可以使用如下函数
    def int_to_date(date_int):
        return datetime.strptime(str(date_int), "%Y%m%d")

    print(f"转换后：最小日期 = {int_to_date(min_date)}, 最大日期 = {int_to_date(max_date)}")

    # 统计唯一的 permno 数量
    permno_count = df.select(pl.col("permno").n_unique()).to_series()[0]
    print(f"\n唯一 permno 数量：{permno_count}")

In [51]:
def count_permno(df):
    # 统计唯一的 permno 数量
    count = df.select(pl.col("permno").n_unique()).to_series()[0]
    print(f"\n唯一 count 数量：{count}")


In [65]:
ret_data = utils.load_data("data/raw_data/return_processed.csv")
data_info(ret_data)
print(ret_data.filter((pl.col('TICKER') == 'WSO') & (pl.col('DATE') == 20220131)))

(793967, 5)

过滤后的数据 DATE 范围（整数格式）：最小值 = 20110131, 最大值 = 20220131
转换后：最小日期 = 2011-01-31 00:00:00, 最大日期 = 2022-01-31 00:00:00

唯一 permno 数量：10734
shape: (2, 5)
┌────────┬──────────┬────────┬────────────┬────────────┐
│ permno ┆ DATE     ┆ TICKER ┆ CUSIP      ┆ RET_target │
│ ---    ┆ ---      ┆ ---    ┆ ---        ┆ ---        │
│ i64    ┆ i64      ┆ str    ┆ f64        ┆ f64        │
╞════════╪══════════╪════════╪════════════╪════════════╡
│ 46068  ┆ 20220131 ┆ WSO    ┆ 9.426221e7 ┆ -0.100403  │
│ 66376  ┆ 20220131 ┆ WSO    ┆ 9.426222e7 ┆ -0.090674  │
└────────┴──────────┴────────┴────────────┴────────────┘


In [73]:
vgnn_data = utils.load_data('data/saved_data.csv')
data_info(vgnn_data)
print(vgnn_data.filter((pl.col('permno') == 46068) & (pl.col('DATE') == 20201231)))
print(vgnn_data.filter((pl.col('permno') == 66376) & (pl.col('DATE') == 20201231)))

(109800, 98)

过滤后的数据 DATE 范围（整数格式）：最小值 = 20180131, 最大值 = 20201231
转换后：最小日期 = 2018-01-31 00:00:00, 最大日期 = 2020-12-31 00:00:00

唯一 permno 数量：3050
shape: (1, 98)
┌──────────┬────────┬──────────┬───────────┬───┬───────────┬───────────┬──────┬────────────┐
│ DATE     ┆ permno ┆ mvel1    ┆ beta      ┆ … ┆ std_turn  ┆ zerotrade ┆ sic2 ┆ RET_target │
│ ---      ┆ ---    ┆ ---      ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---  ┆ ---        │
│ i64      ┆ i64    ┆ f64      ┆ f64       ┆   ┆ f64       ┆ f64       ┆ f64  ┆ f64        │
╞══════════╪════════╪══════════╪═══════════╪═══╪═══════════╪═══════════╪══════╪════════════╡
│ 20201231 ┆ 46068  ┆ 0.191327 ┆ -0.754677 ┆ … ┆ -0.997449 ┆ 0.988946  ┆ 50.0 ┆ 0.054517   │
└──────────┴────────┴──────────┴───────────┴───┴───────────┴───────────┴──────┴────────────┘
shape: (1, 98)
┌──────────┬────────┬──────────┬───────────┬───┬───────────┬───────────┬──────┬────────────┐
│ DATE     ┆ permno ┆ mvel1    ┆ beta      ┆ … ┆ std_turn  ┆ zerotrade ┆ sic2 ┆ RE

In [5]:
# 假设 vgnn_data 和 sub6_data 都是 Polars DataFrame
# 获取小表中唯一的 permno 值
permno_list = vgnn_data['permno'].unique()

# 使用 Polars 的 is_in 方法筛选大表
filtered_ret_df = ret_data.filter(pl.col('permno').is_in(permno_list))

# 查看筛选后大表中唯一 permno 的数量
filtered_permno_count = filtered_ret_df['permno'].n_unique()
print(f"筛选后大表中 permno 的数量: {filtered_permno_count}")


筛选后大表中 permno 的数量: 3050


In [6]:
df_filtered = filtered_ret_df.filter(pl.col("DATE") == 20181031)
print(df_filtered.shape)
result_df = df_filtered.unique(subset=["permno"]).select(["permno", "TICKER"])
print(result_df.shape)
print(result_df.head(10))
null_count = result_df.select(pl.col("TICKER").is_null().sum()).to_series()[0]
print("TICKER 列的 null 值数量:", null_count)


(3050, 5)
(3050, 2)
shape: (10, 2)
┌────────┬────────┐
│ permno ┆ TICKER │
│ ---    ┆ ---    │
│ i64    ┆ str    │
╞════════╪════════╡
│ 92043  ┆ IBKR   │
│ 11293  ┆ WSBC   │
│ 12082  ┆ MMYT   │
│ 89941  ┆ TZOO   │
│ 93073  ┆ ADUS   │
│ 93237  ┆ TENX   │
│ 90989  ┆ CCO    │
│ 85082  ┆ EGP    │
│ 92439  ┆ IFGL   │
│ 17444  ┆ BELFA  │
└────────┴────────┘
TICKER 列的 null 值数量: 0


In [7]:
relation_data = utils.load_data('data/preprocess_data/firms_relation_ticker.csv')

print(relation_data.shape)

# 统计唯一的 permno 数量
count = relation_data.select(pl.col("Exchange:Ticker").n_unique()).to_series()[0]
print(f"\n唯一 count 数量：{count}")

# print(relation_data.head(2))

(5177, 12)

唯一 count 数量：5149


In [23]:
# filtered_ret_df.filter(pl.col("DATE") == 20181031)
flitered_relation = relation_data.filter(pl.col("Exchanges [Primary Listing]") != '-')
# flitered_relation = relation_data.filter(pl.col("Exchange:Ticker") == 'TEL')
print(flitered_relation.head(10))
print(flitered_relation.shape)

shape: (10, 12)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ Company   ┆ Exchange: ┆ Company   ┆ Exchanges ┆ … ┆ Industry  ┆ Headquart ┆ Security  ┆ Excel    │
│ Name      ┆ Ticker    ┆ Type      ┆ [Primary  ┆   ┆ Classific ┆ ers - Cou ┆ Tickers   ┆ Company  │
│ ---       ┆ ---       ┆ ---       ┆ Listing]  ┆   ┆ ations    ┆ ntry/Regi ┆ ---       ┆ ID       │
│ str       ┆ str       ┆ str       ┆ ---       ┆   ┆ ---       ┆ on        ┆ str       ┆ ---      │
│           ┆           ┆           ┆ str       ┆   ┆ str       ┆ ---       ┆           ┆ str      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 1-800-FLO ┆ FLWS      ┆ Public    ┆ Nasdaq    ┆ … ┆ Catalog   ┆ United    ┆ NasdaqGS: ┆ IQ24085  │
│ WERS.COM, ┆           ┆ Company   ┆ Global    ┆   ┆ Flowers,  ┆ States   

In [61]:
print(type(flitered_relation))
print(type(result_df))

print(result_df.filter(pl.col('TICKER') == 'WSO'))

<class 'polars.dataframe.frame.DataFrame'>
<class 'polars.dataframe.frame.DataFrame'>
shape: (2, 2)
┌────────┬────────┐
│ permno ┆ TICKER │
│ ---    ┆ ---    │
│ i64    ┆ str    │
╞════════╪════════╡
│ 66376  ┆ WSO    │
│ 46068  ┆ WSO    │
└────────┴────────┘


In [53]:
merged_df = result_df.join(relation_data, left_on="TICKER", right_on="Exchange:Ticker", how="left")

print(merged_df.shape)
print(merged_df[['TICKER', 'permno']])
count = merged_df.select(pl.col("TICKER").n_unique()).to_series()[0]
print(f"\n唯一 count 数量：{count}")

(3064, 13)
shape: (3_064, 2)
┌────────┬────────┐
│ TICKER ┆ permno │
│ ---    ┆ ---    │
│ str    ┆ i64    │
╞════════╪════════╡
│ IBKR   ┆ 92043  │
│ WSBC   ┆ 11293  │
│ MMYT   ┆ 12082  │
│ TZOO   ┆ 89941  │
│ ADUS   ┆ 93073  │
│ …      ┆ …      │
│ CULP   ┆ 27909  │
│ PRK    ┆ 76266  │
│ DOV    ┆ 25953  │
│ EME    ┆ 82694  │
│ LPX    ┆ 56223  │
└────────┴────────┘

唯一 count 数量：3033


In [57]:
# 假设 df 是你的 DataFrame
duplicates = (
    merged_df.group_by("TICKER")
      .agg(pl.len().alias("count"))
      .filter(pl.col("count") > 1)

)
# 2. 提取重复的 TICKER 列表
dup_tickers = duplicates["TICKER"].to_list()

# 3. 使用 is_in 筛选出原始 DataFrame 中 TICKER 在重复列表中的行
df_duplicates = merged_df.filter(pl.col("TICKER").is_in(dup_tickers))

print(df_duplicates.shape)
print(df_duplicates[['TICKER', 'permno']])


(62, 13)
shape: (62, 2)
┌────────┬────────┐
│ TICKER ┆ permno │
│ ---    ┆ ---    │
│ str    ┆ i64    │
╞════════╪════════╡
│ CCO    ┆ 90989  │
│ CCO    ┆ 90989  │
│ BIO    ┆ 61508  │
│ MKC    ┆ 89155  │
│ STZ    ┆ 64899  │
│ …      ┆ …      │
│ K      ┆ 26825  │
│ HEI    ┆ 85945  │
│ PPL    ┆ 22517  │
│ PPL    ┆ 22517  │
│ TAP    ┆ 59248  │
└────────┴────────┘


In [44]:
# 假设 df 是你的 DataFrame
duplicates = (
    merged_df.group_by("TICKER")
      .agg(pl.len().alias("count"))
      .filter(pl.col("count") > 1)

)
# 2. 提取重复的 TICKER 列表
dup_tickers = duplicates["TICKER"].to_list()

# 3. 使用 is_in 筛选出原始 DataFrame 中 TICKER 在重复列表中的行
df_duplicates = merged_df.filter(pl.col("TICKER").is_in(dup_tickers))

print(df_duplicates)


shape: (62, 13)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬────────┬────────┐
│ Company    ┆ Company    ┆ Exchanges  ┆ Business   ┆ … ┆ Security   ┆ Excel     ┆ permno ┆ TICKER │
│ Name       ┆ Type       ┆ [Primary   ┆ Relationsh ┆   ┆ Tickers    ┆ Company   ┆ ---    ┆ ---    │
│ ---        ┆ ---        ┆ Listing]   ┆ ips (All   ┆   ┆ ---        ┆ ID        ┆ i64    ┆ str    │
│ str        ┆ str        ┆ ---        ┆ Hi…        ┆   ┆ str        ┆ ---       ┆        ┆        │
│            ┆            ┆ str        ┆ ---        ┆   ┆            ┆ str       ┆        ┆        │
│            ┆            ┆            ┆ str        ┆   ┆            ┆           ┆        ┆        │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪════════╪════════╡
│ Cameco Cor ┆ Public     ┆ -          ┆ Ahtium Oyj ┆ … ┆ TSX:CCO;   ┆ IQ379153  ┆ 90989  ┆ CCO    │
│ poration   ┆ Company    ┆            ┆ (Supplier  ┆   ┆ NYSE:CCJ;  ┆     

In [52]:
flitered = df_duplicates.filter(pl.col("Exchanges [Primary Listing]") != '-')
print(flitered.shape)
print(flitered)

count_permno(flitered)

(22, 13)
shape: (22, 13)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬────────┬────────┐
│ Company    ┆ Company    ┆ Exchanges  ┆ Business   ┆ … ┆ Security   ┆ Excel     ┆ permno ┆ TICKER │
│ Name       ┆ Type       ┆ [Primary   ┆ Relationsh ┆   ┆ Tickers    ┆ Company   ┆ ---    ┆ ---    │
│ ---        ┆ ---        ┆ Listing]   ┆ ips (All   ┆   ┆ ---        ┆ ID        ┆ i64    ┆ str    │
│ str        ┆ str        ┆ ---        ┆ Hi…        ┆   ┆ str        ┆ ---       ┆        ┆        │
│            ┆            ┆ str        ┆ ---        ┆   ┆            ┆ str       ┆        ┆        │
│            ┆            ┆            ┆ str        ┆   ┆            ┆           ┆        ┆        │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪════════╪════════╡
│ Bio-Rad    ┆ Public     ┆ New York   ┆ 10x        ┆ … ┆ NYSE:BIO;  ┆ IQ255743  ┆ 61508  ┆ BIO    │
│ Laboratori ┆ Company    ┆ Stock      ┆ Genomics,  ┆   ┆ NYSE:BIO

In [48]:
# 假设 df 是你的 DataFrame
duplicates = (
    flitered.group_by("TICKER")
      .agg(pl.len().alias("count"))
      .filter(pl.col("count") > 1)

)
# 2. 提取重复的 TICKER 列表
dup_tickers = duplicates["TICKER"].to_list()

# 3. 使用 is_in 筛选出原始 DataFrame 中 TICKER 在重复列表中的行
df_duplicates = flitered.filter(pl.col("TICKER").is_in(dup_tickers))

print(df_duplicates)

shape: (22, 13)
┌────────────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬────────┬────────┐
│ Company    ┆ Company    ┆ Exchanges  ┆ Business   ┆ … ┆ Security   ┆ Excel     ┆ permno ┆ TICKER │
│ Name       ┆ Type       ┆ [Primary   ┆ Relationsh ┆   ┆ Tickers    ┆ Company   ┆ ---    ┆ ---    │
│ ---        ┆ ---        ┆ Listing]   ┆ ips (All   ┆   ┆ ---        ┆ ID        ┆ i64    ┆ str    │
│ str        ┆ str        ┆ ---        ┆ Hi…        ┆   ┆ str        ┆ ---       ┆        ┆        │
│            ┆            ┆ str        ┆ ---        ┆   ┆            ┆ str       ┆        ┆        │
│            ┆            ┆            ┆ str        ┆   ┆            ┆           ┆        ┆        │
╞════════════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪════════╪════════╡
│ Bio-Rad    ┆ Public     ┆ New York   ┆ 10x        ┆ … ┆ NYSE:BIO;  ┆ IQ255743  ┆ 61508  ┆ BIO    │
│ Laboratori ┆ Company    ┆ Stock      ┆ Genomics,  ┆   ┆ NYSE:BIO.B ┆     

In [58]:
null_rows = merged_df.filter(pl.col("TICKER") == 'WSO')
print(null_rows)

shape: (2, 13)
┌────────┬────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬───────────┐
│ permno ┆ TICKER ┆ Company    ┆ Company    ┆ … ┆ Industry   ┆ Headquarte ┆ Security   ┆ Excel     │
│ ---    ┆ ---    ┆ Name       ┆ Type       ┆   ┆ Classifica ┆ rs - Count ┆ Tickers    ┆ Company   │
│ i64    ┆ str    ┆ ---        ┆ ---        ┆   ┆ tions      ┆ ry/Region  ┆ ---        ┆ ID        │
│        ┆        ┆ str        ┆ str        ┆   ┆ ---        ┆ ---        ┆ str        ┆ ---       │
│        ┆        ┆            ┆            ┆   ┆ str        ┆ str        ┆            ┆ str       │
╞════════╪════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪═══════════╡
│ 66376  ┆ WSO    ┆ Watsco,    ┆ Public     ┆ … ┆ Capital    ┆ United     ┆ NYSE:WSO;  ┆ IQ313461  │
│        ┆        ┆ Inc.       ┆ Company    ┆   ┆ Goods      ┆ States     ┆ NYSE:WSO.B ┆           │
│        ┆        ┆ (NYSE:WSO) ┆            ┆   ┆ (Primary); ┆            ┆ 